In [ ]:
import pickle

with open('similar_pairs.pkl', 'rb') as file:
    similar_pairs = pickle.load(file)

In [ ]:
similar_pairs[:10], len(similar_pairs)

In [ ]:
import datasets

# Load dataset
dataset = datasets.load_from_disk('./partial_data')

train_split = dataset['train']
train_split[0]

In [ ]:
pair = similar_pairs[9999]
train_split[pair[0]]['images'][0]

In [ ]:
train_split[pair[1]]['images'][0]

In [ ]:
train_split[pair[0]]['findings'], train_split[pair[1]]['findings']

In [ ]:
train_split[pair[0]]['impression'], train_split[pair[1]]['impression']

In [ ]:
from tqdm.notebook import tqdm
from PIL import Image
import multiprocessing
import pickle
from functools import partial

def merge_images(image_a, image_b):
    if image_a.size[0] * image_a.size[1] < image_b.size[0] * image_b.size[1]:
        base = image_b
        overlay = image_a
    else:
        base = image_a
        overlay = image_b

    base = base.convert("RGBA")
    overlay = overlay.convert("RGBA")

    overlay_with_transparency = Image.new("RGBA", overlay.size)
    for x in range(overlay.width):
        for y in range(overlay.height):
            r, g, b, a = overlay.getpixel((x, y))
            overlay_with_transparency.putpixel((x, y), (r, g, b, int(a * 0.5)))

    base.paste(overlay_with_transparency, (0, 0), overlay_with_transparency)
    return base.convert("RGB")

def process_pair(pair, train_split):
    idx_a, idx_b = pair
    image_a = train_split[idx_a]['images'][0]
    image_b = train_split[idx_b]['images'][0]
    findings_a = train_split[idx_a]['findings']
    findings_b = train_split[idx_b]['findings']
    impression_a = train_split[idx_a]['impression']
    impression_b = train_split[idx_b]['impression']
    merged_image = merge_images(image_a, image_b)
    return {
        'source': 'mixgen',
        'images': merged_image,
        'impression_a': impression_a,
        'impression_b': impression_b,
        'findings_a': findings_a,
        'findings_b': findings_b
    }

In [ ]:
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
all_data = list(tqdm(pool.imap(partial(process_pair, train_split=train_split), similar_pairs), total=len(similar_pairs)))
pool.close()
pool.join()

with open('data_with_imgs.pkl', 'wb') as file:
    pickle.dump(all_data, file)

print(len(all_data))